In [ ]:
import matplotlib.pyplot as plt
import matplotlib.style as style 
import matplotlib
import seaborn as sns

# Style
style.use('ggplot')
matplotlib.rcParams['font.family'] = "serif"
sns.set(font='serif')
sns.set_context("poster")  
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)


In [ ]:
import os
import numpy as np
import glob
from metrics.Resnets import cifar_resnet56
import tensorflow as tf

model = cifar_resnet56(load_weights=True)
std = tf.reshape((0.2023, 0.1994, 0.2010), shape=(1, 1, 3))
mean= tf.reshape((0.4914, 0.4822, 0.4465), shape=(1, 1, 3))

directory_path = 'CIFAR_Results'
file_types = ['npy', 'npz']

def write_to_console_file(txt):
  with open(os.path.join(directory_path,'CIFAR_results_classifier.txt'), 'a') as f:
    f.write(txt + '\n')
    print(txt)

def adjust_dynamic_range(data, drange_in, drange_out):
    if drange_in != drange_out:
        scale = (np.float32(drange_out[1]) - np.float32(drange_out[0])) / (np.float32(drange_in[1]) - np.float32(drange_in[0]))
        bias = (np.float32(drange_out[0]) - np.float32(drange_in[0]) * scale)
        data = data * scale + bias
    return data

numpy_vars = {}
for np_name in glob.glob(os.path.join(directory_path,'*.np[yz]')):
  x_test = np.load(np_name).transpose((0,2,3,1))
  x_test_w = adjust_dynamic_range(x_test, [-1, 1], [0, 255])
  x_test_w = np.rint(x_test_w).clip(0, 255).astype(np.uint8)
  x_test_c = tf.cast(x_test_w, tf.float32)/255.
  x_test_c = (x_test_c - mean) / std
  prediction = model(x_test_c)
  prediction = tf.argmax(prediction, axis=-1)
  prediction = prediction.numpy().astype('int8')
  samples_per_mode = np.zeros(10)
  for i in range(10):
      samples_per_mode[i] = np.sum(prediction == i)
  y_pos = np.arange(len(samples_per_mode))

  plt.bar(y_pos,samples_per_mode)
  plt.savefig(np_name[:-4] + '_modes', bbox_inches = 'tight')
  plt.clf()
  plt.close()

  nmbMax = 15
  plt.subplots_adjust(hspace=0, wspace=0)    
  for i in range(nmbMax**2):
    plt.subplot(nmbMax, nmbMax, i + 1)
    plt.imshow(x_test_w[i], aspect='auto')
    plt.axis('off')
  plt.savefig(np_name[:-4] + '_samples', bbox_inches = 'tight')
  plt.close()
  numpy_vars[np_name] = samples_per_mode
  write_to_console_file(np_name + ': {}'.format(samples_per_mode))